In [ ]:
from nsetools import Nse
import pandas as pd
import time
import os
from datetime import datetime, time as dtime, date

nse = Nse()


SYMBOL = input("Enter NSE stock symbol: ").upper()
CSV_FILE = f"{SYMBOL}_live.csv"
INTERVAL = 60


MARKET_START = dtime(9, 15)
MARKET_END = dtime(15, 30)


session_high = None
session_low = None
current_day = date.today()


def is_market_open():
    now = datetime.now().time()
    return MARKET_START <= now <= MARKET_END


def fetch_stock(symbol):
    try:
        quote = nse.get_quote(symbol)
        return quote["lastPrice"]
    except Exception as e:
        print("Error:", e)
        return None


def get_last_price():
    if os.path.exists(CSV_FILE):
        df = pd.read_csv(CSV_FILE)
        if not df.empty:
            return df.iloc[-1]["price"]
    return None


while True:

    if not is_market_open():
        print("Market closed. Waiting for opening....")
        time.sleep(INTERVAL)
        continue
 
    
    if current_day != date.today():
        session_high = None
        session_low = None
        current_day = date.today()

    price = fetch_stock(SYMBOL)

    if price is not None:
        now = datetime.now()

        
        time_only = now.strftime("%H:%M:%S")

      
        last_price = get_last_price()
        if last_price is None:
            movement = "SAME"
        elif price > last_price:
            movement = "UP"
        elif price < last_price:
            movement = "DOWN"
        else:
            movement = "SAME"

       
        if session_high is None:
            session_high = price
            session_low = price
        else:
            session_high = max(session_high, price)
            session_low = min(session_low, price)

        data = {
            "time": time_only,
            "symbol": SYMBOL,
            "price": price,
            "movement": movement,
            "session_high": session_high,
            "session_low": session_low
        }

        df = pd.DataFrame([data])

        if os.path.exists(CSV_FILE):
            df.to_csv(CSV_FILE, mode="a", header=False, index=False)
        else:
            df.to_csv(CSV_FILE, mode="w", header=True, index=False)

        print("Saved:", data)

    time.sleep(INTERVAL)
